# Building an Analytics Dashboard Assistant with OpenAI

This tutorial will guide you through creating an intelligent analytics assistant using OpenAI's Assistants API. Our assistant will be capable of:
- Analyzing multiple data files using File Search
- Generating visualizations and insights using Code Interpreter
- Creating interactive dashboards based on user queries

## Setup and Dependencies

First, let's install the required packages:

In [ ]:
%pip install openai pandas matplotlib seaborn plotly

In [4]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [6]:
import json
import time
from IPython.display import display, HTML

## Initializing the OpenAI Client

First, we'll set up our OpenAI client with the appropriate API key:

In [8]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

## Creating the Analytics Assistant

We'll create an assistant that combines both Code Interpreter and File Search capabilities:

In [9]:
def create_analytics_assistant():
    assistant = client.beta.assistants.create(
        name="Analytics Dashboard Assistant",
        instructions="""You are an expert data analyst and visualization specialist. 
        Your role is to:
        1. Analyze data files provided by users
        2. Generate insightful visualizations
        3. Create comprehensive analytics dashboards
        4. Explain trends and patterns in the data
        Always provide clear explanations of your analysis process.""",
        model="gpt-4o",
        tools=[
            {"type": "code_interpreter"},
            {"type": "file_search"}
        ]
    )
    return assistant

analytics_assistant = create_analytics_assistant()

## Setting Up the Vector Store for File Search

The File Search capability requires setting up a vector store for our data files:

In [11]:
def create_vector_store(name="Analytics Files"):
    vector_store = client.beta.vector_stores.create(
        name=name,
    )
    return vector_store

def add_files_to_vector_store(vector_store_id, file_ids):
    batch = client.beta.vector_stores.file_batches.create_and_poll(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    )
    return batch

# Create vector store
vector_store = create_vector_store()

# Update assistant with vector store
analytics_assistant = client.beta.assistants.update(
    assistant_id=analytics_assistant.id,
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

## File Upload Helper Functions

Let's create helper functions to handle file uploads:

In [12]:
def upload_file(file_path):
    """Upload a file for the assistant to use"""
    with open(file_path, 'rb') as file:
        response = client.files.create(
            file=file,
            purpose='assistants'
        )
    return response

def attach_files_to_assistant(assistant_id, file_ids):
    """Attach files to the assistant for code interpreter"""
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        tool_resources={
            "code_interpreter": {
                "file_ids": file_ids
            }
        }
    )
    return assistant

## Creating and Managing Threads

Now let's create functions to manage conversation threads:

In [13]:
def create_thread_with_files(files=None):
    """Create a new thread with optional files"""
    if files:
        messages = [{
            "role": "user",
            "content": "I've uploaded some files for analysis.",
            "attachments": [
                {
                    "file_id": file_id,
                    "tools": [{"type": "code_interpreter"}, {"type": "file_search"}]
                } for file_id in files
            ]
        }]
        thread = client.beta.threads.create(messages=messages)
    else:
        thread = client.beta.threads.create()
    return thread

def add_message_to_thread(thread_id, content, files=None):
    """Add a message to an existing thread"""
    if files:
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=content,
            attachments=[
                {
                    "file_id": file_id,
                    "tools": [{"type": "code_interpreter"}, {"type": "file_search"}]
                } for file_id in files
            ]
        )
    else:
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=content
        )
    return message

## Running the Assistant and Handling Responses

Here's how we'll handle running the assistant and processing its responses:

In [14]:
def run_assistant(thread_id, assistant_id):
    """Create and manage a run of the assistant"""
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )
        
        if run.status == 'completed':
            break
        elif run.status == 'failed':
            raise Exception(f"Run failed: {run.last_error}")
        elif run.status == 'requires_action':
            # Handle any required actions (function calls, etc.)
            pass
        
        time.sleep(1)
    
    # Get messages after run completes
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return messages

def display_assistant_response(messages):
    """Display the assistant's response including any generated visualizations"""
    for message in messages:
        if message.role == "assistant":
            for content in message.content:
                if content.type == 'text':
                    print(content.text.value)
                elif content.type == 'image_file':
                    # Handle image display
                    file_id = content.image_file.file_id
                    image_data = client.files.content(file_id)
                    # Display image using IPython
                    display(HTML(f'<img src="data:image/png;base64,{image_data}" />'))

## Example Usage: Creating an Analytics Dashboard

Let's put it all together with an example:

In [18]:
# Upload sample data files
sales_data = upload_file('sales_data.csv')

# # Add files to vector store
# add_files_to_vector_store(vector_store.id, [sales_data.id])

# Create a thread with the files
thread = create_thread_with_files([sales_data.id])

# Ask for analysis and dashboard creation
analysis_request = """
Please analyze the sales and customer data to create a dashboard that shows:
1. Monthly sales trends
2. Customer segmentation by purchase value
3. Top-performing products
4. Customer retention rates

Generate visualizations and provide insights for each metric.
"""

add_message_to_thread(thread.id, analysis_request)

# Run the assistant
messages = run_assistant(thread.id, analytics_assistant.id)

# Display the results
display_assistant_response(messages)

BadRequestError: Error code: 400 - {'error': {'message': 'Files with extensions [.csv] are not supported for retrieval. See https://platform.openai.com/docs/assistants/tools/file-search/supported-files', 'type': 'invalid_request_error', 'param': None, 'code': 'unsupported_file'}}

## Best Practices and Tips

1. **File Management**:
   - Keep track of file IDs and clean up unused files
   - Use appropriate file formats (CSV, JSON, Excel) for data
   - Consider file size limits (512MB per file)

2. **Vector Store Organization**:
   - Group related files in the same vector store
   - Use descriptive names for vector stores
   - Monitor vector store expiration policies

3. **Error Handling**:
   - Implement proper error handling for API calls
   - Monitor run status and handle failures gracefully
   - Validate file uploads and data formats

4. **Performance Optimization**:
   - Use appropriate chunk sizes for File Search
   - Monitor token usage and context windows
   - Implement request rate limiting

## Conclusion

This tutorial demonstrated how to create an intelligent analytics assistant that combines the power of OpenAI's Code Interpreter and File Search capabilities. The assistant can analyze multiple data sources, generate visualizations, and create interactive dashboards based on user queries.

You can extend this foundation by:
- Adding more sophisticated visualization capabilities
- Implementing custom dashboard templates
- Adding support for more data formats
- Creating specialized analysis functions
- Implementing caching for frequently accessed data

Remember to handle API keys securely and implement proper error handling in production environments.